In [1]:
import time
import re
import requests
import numpy as np
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from IPython.core.display import display, HTML

In [9]:
headers = {
    "Connection" : "keep-alive",
    "Cache-Control" : "max-age=0",
    "sec-ch-ua-mobile" : "?0",
    "DNT" : "1",
    "Upgrade-Insecure-Requests" : "1",
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site" : "none",
    "Sec-Fetch-Mode" : "navigate",
    "Sec-Fetch-User" : "?1",
    "Sec-Fetch-Dest" : "document",
    "Accept-Encoding" : "gzip, deflate, br",
    "Accept-Language" : "ko-KR,ko;q=0.9"
    }
 

def dc_url_crawler(keyword,page_num):
    url_list = []
    date_list = []
    for page in range(page_num+1):
        url = f"https://search.dcinside.com/post/p/{page}/sort/accuracy/q/{keyword}"

        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml")

        result_list = soup.find_all("ul",class_='sch_result_list')
        for i in result_list:
            for j in i.find_all('span',class_='date_time'):
                date = j.text
                date_list.append(date)
            href = i.find_all('a',class_='tit_txt')
            for h in href:
                url = h.attrs['href']
                url_list.append(url)
        time.sleep(3)
    drop_date = [index for index, date in enumerate(date_list) if date <= '2019.01.01 00:00:00']
    new_date = np.delete(date_list, drop_date, axis=0)
    new_url = np.delete(url_list, drop_date, axis=0)
    return new_url, new_date

keyword = "산업공학과"
 
new_url, new_date = dc_url_crawler(keyword,2)

In [ ]:
def dc_cralwer(url_list):
    title_list = []
    content_list = []
    for url in url_list:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml")
        title = soup.find('span',class_='title_subject').text
        print(title)
        content = soup.find('div',class_='write_div')
        print(content.text)
        title_list.append(title)
        content_list.append(content)
    df = pd.DataFrame({'title':title_list,'content':content})
    return df
